In [5]:
import os
import numpy as np
import random
import cv2
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [6]:
DATASET_DIR = ("/content/drive/MyDrive/HandGesture/images")
LABELS = os.listdir(DATASET_DIR)
IMAGE_SIZE = (224, 224)

IMAGES = []
for label in LABELS:
    label_dir = os.path.join(DATASET_DIR, label)
    for img in tqdm(os.listdir(label_dir), desc=f"Processing images for '{label}' --->"):
        img_np = cv2.imread(os.path.join(label_dir, img))
        img_np = cv2.resize(img_np, IMAGE_SIZE)
        IMAGES.append({
            "img": img_np,
            "label": label,
            "label_num": LABELS.index(label)
        })


Processing images for 'call_me' --->: 100%|██████████| 527/527 [00:05<00:00, 87.88it/s] 


In [7]:
random.shuffle(IMAGES)

In [8]:
X_IMAGES, y_IMAGES = train_test_split(IMAGES, test_size=0.2, random_state=42)
X_train = np.array([img["img"] for img in X_IMAGES])
y_train = np.array([img["label_num"] for img in X_IMAGES])
X_test = np.array([img["img"] for img in y_IMAGES])
y_test = np.array([img["label_num"] for img in y_IMAGES])

In [ ]:
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)
y_train_encoded = to_categorical(y_train, num_classes=len(LABELS))
y_test_encoded = to_categorical(y_test, num_classes=len(LABELS))

In [9]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))
base_model.trainable = False  # Freeze base layers

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(len(LABELS), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train_encoded, epochs=10, validation_data=(X_test, y_test_encoded))

Epoch 1/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.6391 - loss: 1.1177 - val_accuracy: 0.9800 - val_loss: 0.0897
Epoch 2/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - accuracy: 0.9518 - loss: 0.1840 - val_accuracy: 0.9933 - val_loss: 0.0430
Epoch 3/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.9682 - loss: 0.1153 - val_accuracy: 0.9943 - val_loss: 0.0238
Epoch 4/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 241s 1s/step - accuracy: 0.9788 - loss: 0.0820 - val_accuracy: 0.9971 - val_loss: 0.0175
Epoch 5/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.9788 - loss: 0.0714 - val_accuracy: 0.9952 - val_loss: 0.0197
Epoch 6/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - accuracy: 0.9732 - loss: 0.0848 - val_accuracy: 0.9943 - val_loss: 0.0158
Epoch 7/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.9902 - loss: 0.0428 - val_accuracy: 0.9971 - val_loss: 0.0144
Epoch 8/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.9862 - loss: 0.0405 - val_accu

In [11]:
predictions = model.predict(X_test)
pred = np.argmax(predictions, axis=1)
accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)

33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step
Accuracy: 0.998093422306959


In [12]:
report = classification_report(y_test, pred, target_names=LABELS)
print(report)

                 precision    recall  f1-score   support

          peace       0.99      1.00      1.00       114
        rock_on       1.00      1.00      1.00       117
fingers_crossed       1.00      0.99      1.00       104
             up       1.00      1.00      1.00        88
         thumbs       1.00      1.00      1.00       108
           rock       1.00      1.00      1.00       100
        scissor       1.00      1.00      1.00       113
          paper       0.99      1.00      1.00       107
           okay       1.00      0.99      0.99        94
        call_me       1.00      1.00      1.00       104

       accuracy                           1.00      1049
      macro avg       1.00      1.00      1.00      1049
   weighted avg       1.00      1.00      1.00      1049

